# Imports / Configuration

In [29]:
import numpy as np
import time

import sys,os

data_path = os.getcwd()

try:
    import localgraphclustering as lgc
except:
    # when the package is not installed, import the local version instead. 
    # the notebook must be placed in the original "notebooks/" folder
    sys.path.append("../")
    import localgraphclustering as lgc

## Load dataset

In [30]:
g = lgc.GraphLocal(os.path.join(data_path,'datasets/JohnsHopkins.graphml'),'graphml')

## Define proximal accelerated gradient descent function

In [31]:
def proximal_accel_gradient_descent(A, d, ref_node, rho, alpha, eps, stepsize=1.):
    
    # Number of nodes in the graph
    n = d.shape[0]
    
    # Initialize seed vector
    seed = np.zeros(n)
    seed[ref_node] = 1
    
    # Initialized paramters
    x = np.zeros(n)

    # Initiliaze algorithm statistics
    err = 100.
    ite = 0
    
    # Another vector that we will need
    y = x
    
    # Compute gradient
    grad = ((1+alpha)/2)*(d*y)  - ((1-alpha)/2)*(A@y) - alpha * seed

    # The algorithm starts here
    while err > eps:
        
        # beta is a parameter for the accelerated algorithm
        if ite == 0:
            beta = 0
        else:
            beta = (1-np.sqrt(alpha)/(1+np.sqrt(alpha)))
            
        # Update parameters using a gradient step
        z = y - stepsize * grad / d 
        
        # Store old parameters
        x_old = x
        
        # Update parameters using the proximal step
        x = np.sign(z) * np.maximum(abs(z) - stepsize * rho * alpha, 0)
                
        # Update parameters y
        y = x + beta*(x - x_old)
        
        # Compute gradient
        grad = ((1+alpha)/2)*(d*y)  - ((1-alpha)/2)*(A@y) - alpha * seed
    
        # Compute termination criterion
        err = np.linalg.norm(x_old - x, np.inf)
        
        # Increase iteration count
        ite += 1
        
    return x

## Run proximal accelerated gradient descent on the givne graph

In [32]:
# L1-regularization parameter for the spectral problem
rho = 8.0e-5
# Teleportation parameter of the PageRank model
alpha = 0.1
# Refence node
ref_node = 2338

# Matrices and vectors that are needed by the present implementation of proximal gradient descent
A = g.adjacency_matrix.tocsc()
d = g.d

# Stepsize for algorithm
stepsize = 1
# Termination tolerance
epsilon = 1.0e-10

start2 = time.time()
x = proximal_accel_gradient_descent(A, d, ref_node, rho, alpha, epsilon, stepsize = stepsize)
end2 = time.time()
print(end2 - start2)

0.051496267318725586


## Print output

In [33]:
np.nonzero(x)[0]

array([  65,  263,  505,  842,  857, 1139, 1320, 1380, 1412, 1516, 1756,
       1788, 1836, 1978, 2255, 2276, 2320, 2338, 2352, 2366, 2549, 2569,
       2846, 2932, 2981, 3084, 3326, 3392, 3428, 3583, 3641, 3891, 3929,
       4221, 4227, 4287, 4332, 4395, 4516, 4521, 4550, 4558, 4637, 4856,
       4866, 4879, 5004, 5031, 5032, 5127])

In [34]:
x[np.nonzero(x)[0]]

array([1.59695610e-05, 3.41660320e-06, 6.24787392e-04, 2.35048727e-06,
       1.47418550e-05, 2.77244067e-04, 3.71913389e-06, 9.66331410e-06,
       3.03001487e-06, 1.19307773e-05, 4.96644110e-04, 1.56326570e-04,
       1.06497636e-04, 3.75127530e-05, 3.53921382e-06, 1.91281042e-05,
       2.17312065e-04, 1.04159639e-01, 4.21635591e-05, 1.90708614e-05,
       4.26755925e-06, 1.57955281e-05, 1.97594899e-05, 4.59760906e-05,
       2.78886037e-06, 2.80860364e-05, 3.36712595e-06, 2.04484250e-05,
       1.27428948e-05, 1.00781713e-05, 4.13194660e-07, 3.23462303e-06,
       1.67074871e-03, 5.63885848e-05, 3.23306566e-05, 1.17686376e-05,
       2.77143335e-05, 2.85909339e-06, 2.96148937e-02, 2.81088954e-03,
       4.55057868e-05, 2.63051717e-05, 3.56966266e-05, 1.76827896e-04,
       4.72011032e-05, 5.54775131e-06, 3.39747912e-05, 4.54990569e-06,
       6.15105050e-06, 1.37612666e-05])

## Define proximal gradient descent

In [35]:
def proximal_gradient_descent(A, d, ref_node, rho, alpha, eps, stepsize=1.):
    
    # Number of nodes in the graph
    n = d.shape[0]
    
    # Initialize seed vector
    seed = np.zeros(n)
    seed[ref_node] = 1
    
    # Initialized paramters
    x = np.zeros(n)

    # Initiliaze algorithm statistics
    err = 100.
    ite = 0

    # The algorithm starts here
    while err > eps:
        # Store old parameters
        x_old = x
        # Compute gradient
        grad = ((1+alpha)/2)*(d*x)  - ((1-alpha)/2)*(A@x) - alpha * seed
        # Update parameters using a gradient step
        x = x - stepsize * grad / d 
        # Update parameters using the proximal step
        x = np.sign(x) * np.maximum(abs(x) - stepsize * rho * alpha, 0)
        # Compute termination criterion
        err = np.linalg.norm(x_old - x, np.inf)
        # Increase iteration count
        ite += 1
        
    return x

## Run proximal gradient descent

In [36]:
# L1-regularization parameter for the spectral problem
rho = 8.0e-5
# Teleportation parameter of the PageRank model
alpha = 0.1
# Refence node
ref_node = 2338

# Matrices and vectors that are needed by the present implementation of proximal gradient descent
A = g.adjacency_matrix
d = g.d

# Stepsize for algorithm
stepsize = 1
# Termination tolerance
epsilon = 1.0e-10

# Call proximal gradient descent
x = proximal_gradient_descent(A, d, ref_node, rho, alpha, epsilon, stepsize = stepsize)

In [37]:
np.nonzero(x)[0]

array([  65,  263,  505,  842,  857, 1139, 1320, 1380, 1412, 1516, 1756,
       1788, 1836, 1978, 2255, 2276, 2320, 2338, 2352, 2366, 2549, 2569,
       2846, 2932, 2981, 3084, 3326, 3392, 3428, 3583, 3641, 3891, 3929,
       4221, 4227, 4287, 4332, 4395, 4516, 4521, 4550, 4558, 4637, 4856,
       4866, 4879, 5004, 5031, 5032, 5127])

In [38]:
x[np.nonzero(x)[0]]

array([1.59695609e-05, 3.41660317e-06, 6.24787394e-04, 2.35048724e-06,
       1.47418549e-05, 2.77244067e-04, 3.71913386e-06, 9.66331406e-06,
       3.03001484e-06, 1.19307773e-05, 4.96644114e-04, 1.56326570e-04,
       1.06497636e-04, 3.75127529e-05, 3.53921387e-06, 1.91281042e-05,
       2.17312065e-04, 1.04159639e-01, 4.21635590e-05, 1.90708614e-05,
       4.26755922e-06, 1.57955281e-05, 1.97594899e-05, 4.59760905e-05,
       2.78886035e-06, 2.80860363e-05, 3.36712593e-06, 2.04484250e-05,
       1.27428947e-05, 1.00781713e-05, 4.13194636e-07, 3.23462294e-06,
       1.67074871e-03, 5.63885847e-05, 3.23306565e-05, 1.17686376e-05,
       2.77143335e-05, 2.85909337e-06, 2.96148937e-02, 2.81088954e-03,
       4.55057867e-05, 2.63051715e-05, 3.56966265e-05, 1.76827896e-04,
       4.72011031e-05, 5.54775128e-06, 3.39747911e-05, 4.54990567e-06,
       6.15105047e-06, 1.37612666e-05])